In [41]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np
import random

In [42]:
def warp(im, src, target):
    h, status = cv2.findHomography(src, target)
    return cv2.warpPerspective(im, h, (im.shape[1],im.shape[0]))


def rotate_box(bb, cx, cy, h, w, angle):
    new_bb = list(bb)
    for i,coord in enumerate(bb):
        # opencv calculates standard transformation matrix
        M = cv2.getRotationMatrix2D((cx, cy), angle, 1.0)
        # Grab  the rotation components of the matrix)
        cos = np.abs(M[0, 0])
        sin = np.abs(M[0, 1])
        # compute the new bounding dimensions of the image
        nW = int((h * sin) + (w * cos))
        nH = int((h * cos) + (w * sin))
        # adjust the rotation matrix to take into account translation
        M[0, 2] += (nW / 2) - cx
        M[1, 2] += (nH / 2) - cy
        # Prepare the vector to be transformed
        v = [coord[0],coord[1],1]
        # Perform the actual rotation and return the image
        calculated = np.dot(M,v)
        new_bb[i] = (calculated[0],calculated[1])
    return new_bb


def rotate(image, src, amount):
    angle = amount
    # grab the dimensions of the image and then determine the
    # centre
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    (heigth, width) = image.shape[:2]
    (cx, cy) = (width // 2, heigth // 2)
    ar = rotate_box(src, cx, cy, heigth, width, angle)
    l = []
    for t in ar:
        l.append(np.asarray(t))

    return cv2.warpAffine(image, M, (nW, nH)), np.asarray(l).astype(int)


def offset(image, src, amount, axis):
    pts_dst = src.copy()
    for pt in pts_dst:
        pt[axis]+=amount
    return warp(image, src, pts_dst), pts_dst


def resize(image, src, amount, axis):
    pts_dst = src.copy()
    for pt in pts_dst:
        pt[axis] *= amount
    return warp(image, src, pts_dst), pts_dst

def offset_x(image, src, amount):
    return offset(image, src, amount, 0)

def offset_y(image, src, amount):
    return offset(image, src, amount, 1)

def resize_x(image, src, amount):
    return resize(image, src, amount, 0)

def resize_y(image, src, amount):
    return resize(image, src, amount, 1)



In [71]:
def rand_transform(image, origin):
    image, origin = rotate(image, origin, random.randrange(0, 360))
    image, origin = offset_x(image, origin, random.randrange(-30, 30))
    image, origin = offset_y(image, origin, random.randrange(-30, 30))
    image, origin = resize_x(image, origin, random.randrange(-20, 20) / 100 + 1)
    image, origin = resize_y(image, origin, random.randrange(-20, 20) / 100 + 1)
    return image, origin


In [78]:
for dirPath, dirNames, fileNames in os.walk("../sample-puzzles"):
    idx = 0
    for f in fileNames:
        print(++idx)
        fname = os.path.splitext(f)[0]
        im_src = cv2.imread(os.path.join(dirPath, f))
        pts_src = np.array([[128, 128], [256, 128], [256, 256],[128, 256]])
        im_src, pts_src = rand_transform(im_src, pts_src)

        nppts = np.reshape(pts_src, 8)
        nw, nh = im_src.shape[0], im_src.shape[1]
        diffX = int((nw - 384) / 2)
        diffY = int((nh - 384) / 2)

        ul = diffX, diffY
        ur = diffX + 384, diffY
        br = diffX + 384, diffY + 384
        bl = diffX, diffY + 384

        im_src = im_src[diffY:diffY+384, diffX:diffX+384]
        cv2.imwrite("../train-data/images/{0}.jpg".format(fname), im_src)

        for pt in pts_src:
            pt[0] -= diffX
            pt[1] -= diffY
            cv2.circle(im_src, tuple(pt), 4, (255,255,255))
        cv2.imwrite("../train-data/point-validation/{0}.jpg".format(fname), im_src)
        with open("../train-data/labels/{0}.txt".format(fname), 'w') as out:
             np.savetxt(out,  nppts / 384)


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [77]:
with open("../train-data/labels/20200714_192055.txt", 'r') as re:
    pt = np.genfromtxt(re)
    print(pt)
    print((pt * 384))
    pt = (pt.reshape(4, 2) * 384).astype(int)
    print(pt)
    im = cv2.imread("../train-data/images/20200714_192055.jpg")
    for p in pt:
        cv2.circle(im, tuple(p), 4, (255,255,255))
    cv2.imwrite("../train-data/images/v.jpg", im)



[0.30989583 0.4453125  0.6796875  0.328125   0.80989583 0.66666667
 0.4375     0.78645833]
[119. 171. 261. 126. 311. 256. 168. 302.]
[[119 171]
 [261 126]
 [311 256]
 [168 302]]
